<a href="https://colab.research.google.com/github/Luca-Skyline/check4mate/blob/main/Chess_Piece_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

print(tf.__version__)

2.13.0


In [5]:
!git clone https://github.com/Luca-Skyline/check4mate/tree/main/Chess_Classification_Dataset
%cd /content/check4mate/Chess_Classification_Dataset

/content/check4mate/Chess_Classification_Dataset


In [11]:
!ls

README.dataset.txt  README.roboflow.txt  train	train.zip  valid


In [20]:
train_images = []
train_labels = []

for class_index, folder in enumerate(os.listdir('train/')):
  current_folder_path = os.path.join('train/', folder)
  if os.path.isdir(current_folder_path):
    train_images.append([])
    for image in os.listdir(current_folder_path):
      current_image_path = os.path.join(current_folder_path, image)
      if os.path.isfile(current_image_path):
        image = cv2.imread(current_image_path, cv2.IMREAD_GRAYSCALE)
        pixel_array = np.array(image.tolist())
        train_images[class_index].append(pixel_array)
    train_labels.append(class_index)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [21]:
test_images = []
test_labels = []

for class_index, folder in enumerate(os.listdir('valid/')):
  current_folder_path = os.path.join('valid/', folder)
  if os.path.isdir(current_folder_path):
    test_images.append([])
    for image in os.listdir(current_folder_path):
      current_image_path = os.path.join(current_folder_path, image)
      if os.path.isfile(current_image_path):
        image = cv2.imread(current_image_path, cv2.IMREAD_GRAYSCALE)
        pixel_array = np.array(image.tolist())
        test_images.append(pixel_array)
    test_labels.append(class_index)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

KeyboardInterrupt: ignored

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays,
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

'multiclass'

In [ ]:
dataset.location

'/content/Chess-Piece-Detector-1'